<a href="https://colab.research.google.com/github/sukantamitra007/GenAILearning/blob/TransformerModel/Project_Work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Prepare Dataset for Fine Tunning
1. Remove all review which having review comment as 2 to remove the baise

Models for text summarization
  summarization in few-shot settings T5

 Evaltution metrics is the ROUGE score (short for Recall-Oriented Understudy for Gisting Evaluation).

In [ ]:
pip install gradio

In [ ]:
import gradio as gr
def greet(name):
    return "Hello " + name
demo = gr.Interface(fn=greet, inputs="text", outputs="text")
demo.launch()

# Lets Install Necessary libraries

In [6]:
pip install transformers datasets evaluate rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.0 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=cf4fece49cf08b5d97527fb3e9c7e7d5a07db7d665fdbb254d422501fc16a91c
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
